## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df["APPLICATION_TYPE"].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 100
application_types_to_replace = application_counts[application_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_classification_counts = classification_counts[classification_counts > 1]
print(filtered_classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Check the number of unique values in ASK_AMT column
unique_values = application_df['ASK_AMT'].unique()
num_unique_values = len(unique_values)
print("Unique values:", unique_values)
print(num_unique_values)

Unique values: [    5000   108590     6692 ...     6713    20197 36500179]
8747


In [10]:
# Define bin edges manually based on your data
bin_edges = [0, 5000, 8000, 10000000]  # Adjust these edges as necessary
bin_labels = ["Low", "Medium", "High"]

# Create bins using pd.cut()
application_df['ASK_AMT_Binned'] = pd.cut(application_df['ASK_AMT'], bins=bin_edges, labels=bin_labels, include_lowest=True)

application_df.head(10)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_AMT_Binned
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,Low
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,High
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,Low
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,Medium
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,High
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1,Low
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1,High
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1,High
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1,High
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,Low


In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df[["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE","ORGANIZATION", "STATUS","INCOME_AMT", "SPECIAL_CONSIDERATIONS", "ASK_AMT_Binned", "IS_SUCCESSFUL"]], dtype=int)

application_dummies_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_Binned_Low,ASK_AMT_Binned_Medium,ASK_AMT_Binned_High
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_dummies_df["IS_SUCCESSFUL"].values
X = application_dummies_df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
num_columns = len(application_dummies_df.columns)
print("Number of columns:", num_columns)

Number of columns: 47


## Compile, Train and Evaluate the Model

In [15]:
## Model Optimization Attemp 1

In [16]:
# My first attempt at optimizing the model includes the preprocessing step of binning the ASK_AMT column.

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=46))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,221 (24.30 KB)

 Trainable params: 6,221 (24.30 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 302us/step - accuracy: 0.7067 - loss: 0.5881
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 286us/step - accuracy: 0.7325 - loss: 0.5529
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step - accuracy: 0.7309 - loss: 0.5473
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 284us/step - accuracy: 0.7289 - loss: 0.5527
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 283us/step - accuracy: 0.7366 - loss: 0.5414
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step - accuracy: 0.7323 - loss: 0.5475
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 283us/step - accuracy: 0.7390 - loss: 0.5387
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step - accuracy: 0.7320 - loss: 0.5418
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step - accuracy: 0.7316 - loss: 0.5478
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step - accuracy: 0.7347 - loss: 0.5435
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 293us/step - accuracy: 0.7290 - loss: 0.5469
Epoch 12/100
804/80

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 382us/step - accuracy: 0.7304 - loss: 0.5659
Loss: 0.5658702850341797, Accuracy: 0.7303789854049683


In [21]:
# Export our model to HDF5 file
nn.save('first_model.h5')

In [22]:
## Model Optimization Attempt 2

In [23]:
# My second attempt at optimizing the model includes adding a hidden layer and using the same preprocessing as above.

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=46))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 80)             │         3,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │           465 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,671 (26.06 KB)

 Trainable params: 6,671 (26.06 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 302us/step - accuracy: 0.7014 - loss: 0.5948
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.7308 - loss: 0.5543
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.7328 - loss: 0.5500
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 294us/step - accuracy: 0.7315 - loss: 0.5494
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.7327 - loss: 0.5459
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 297us/step - accuracy: 0.7343 - loss: 0.5457
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 296us/step - accuracy: 0.7349 - loss: 0.5440
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 297us/step - accuracy: 0.7286 - loss: 0.5460
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.7338 - loss: 0.5450
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 299us/step - accuracy: 0.7362 - loss: 0.5399
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 294us/step - accuracy: 0.7331 - loss: 0.5425
Epoch 12/100
804/80

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 398us/step - accuracy: 0.7269 - loss: 0.5708
Loss: 0.5708458423614502, Accuracy: 0.7268804907798767


In [28]:
# Export our model to HDF5 file
nn.save('second_model.h5')

In [29]:
## Model Optimization Attempt 3

In [30]:
# My third attempt at optimizing the model includes changing the number of epochs and using the same preprocessing as above.

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=46))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 80)             │         3,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,221 (24.30 KB)

 Trainable params: 6,221 (24.30 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=75)

Epoch 1/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step - accuracy: 0.7072 - loss: 0.5859
Epoch 2/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 290us/step - accuracy: 0.7266 - loss: 0.5575
Epoch 3/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.7243 - loss: 0.5535
Epoch 4/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 288us/step - accuracy: 0.7349 - loss: 0.5470
Epoch 5/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 286us/step - accuracy: 0.7324 - loss: 0.5478
Epoch 6/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.7351 - loss: 0.5441
Epoch 7/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 285us/step - accuracy: 0.7328 - loss: 0.5487
Epoch 8/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 284us/step - accuracy: 0.7366 - loss: 0.5423
Epoch 9/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - accuracy: 0.7343 - loss: 0.5420
Epoch 10/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.7334 - loss: 0.5469
Epoch 11/75
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.7305 - loss: 0.5475
Epoch 12/75
804/804 ━━━━━━━━━━

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 389us/step - accuracy: 0.7276 - loss: 0.5630
Loss: 0.5630072355270386, Accuracy: 0.727580189704895


In [35]:
# Export our model to HDF5 file
nn.save('third_model.h5')

In [36]:
#I was not able to achieve model accuracy of at least 75%.